In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
# from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
%matplotlib inline




In [ ]:
round(event.muon.tel[1].efficiency.optical_efficiency,3)

In [ ]:
source = EventSource("/Users/vdk/Software/ctasoft/calibpipe/src/calibpipe/tests/data/throughput/simtel_run501_muon_telescope_transmission_0.8.simtel.gz", max_events = 100)
event_iterator = iter(source)
for i,event in enumerate(event_iterator):
    image_processor = ImageProcessor(source.subarray)
    muon_processor = MuonProcessor(source.subarray)
    calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
    calib(event)
    image_processor(event)
    muon_processor(event)
    plt.figure(figsize = (9,6))
    camgeom = source.subarray.tel[1].camera.geometry
    title = f'event {event.index.event_id} in telescope with opt_eff {round(event.muon.tel[1].efficiency.optical_efficiency,3)}'
    disp = CameraDisplay(camgeom,title=title)
    #disp.image = event.r1.tel[m].waveform.sum(axis = 1)
    disp.image = event.dl1.tel[1].image
    #disp.cmap = plt.cm.viridis
    disp.cmap = plt.cm.RdBu_r
    disp.add_colorbar()
    disp.set_limits_percent(95)
    plt.show()

In [ ]:
source = EventSource("/Users/vdk/Software/ctasoft/calibpipe/src/calibpipe/tests/data/throughput/simtel_run501_muon_telescope_transmission_0.8.simtel.gz", max_events = 100)
event_iterator = iter(source)
event = next(event_iterator)



In [ ]:
source.subarray.tels[1].optics_name

In [ ]:
import numpy as np

filename1 = '/Users/vdk/energies_500.txt'
fi1 = open(filename1,'r')
lines = fi1.readlines() 
splitted = lines[0].split(',')
energies_proton_1 = np.array(list(filter(None, splitted))).astype('float')

filename2 = '/Users/vdk/energies_501-1100.txt'
fi2 = open(filename2,'r')
lines2 = fi2.readlines() 
splitted2 = lines2[0].split(',')
energies_proton_2 = np.array(list(filter(None, splitted2))).astype('float')

filename2 = '/Users/vdk/energies_for_filerange1-1100.txt'
fi2 = open(filename2, 'r')
lines = fi2.readlines()
energies_muon = np.array(lines).astype('float')

w_protons = sum([energy**(-0.7) for energy in energies_proton_1])
w_protons = w_protons + sum([energy**(-0.7) for energy in energies_proton_2])
#new_proton_sample = len(energies_proton)/w_protons * len(energies_proton)
new_proton_sample = w_protons
print("Initial proton sample size = ", len(energies_proton_1))
print("Weighted proton sample size = ", new_proton_sample)


w_muons = sum([energy**(-0.7) for energy in energies_muon])
#new_muon_sample = len(energies_muon)/w_muons * len(energies_muon)
new_muon_sample = w_muons
print("Initial muon sample size = ", len(energies_muon))
print("Weighted muon sample size = ", new_muon_sample)

print("Initial relation between muon sample size and proton sample = ", len(energies_muon)/(len(energies_proton_1) + len(energies_proton_2)))
print("Weighed relation between muon sample size and proton sample = ", new_muon_sample / new_proton_sample)

In [ ]:
print(1112/864648)
print(9124/(9e6*0.79))

In [ ]:
event_container = {}
waveform_min_amplitude = 2
min_pixel = 200
pixel_number_arr = np.array([])
pixel_number_dict = {}

for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=10)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        if len(event.trigger.tels_with_trigger) > 1:
            for tel_id in event.trigger.tels_with_trigger:
                tmp_arr = np.array([])
                for pix_id in event.r1.tel[tel_id].waveform:
                    #print("pix_id = ", pix_id)
                    #print("max pix_id = ", np.max(pix_id))
                    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                print("tmp_arr = ", tmp_arr)
                print("len tmp_arr = ", len(tmp_arr[tmp_arr>waveform_min_amplitude]))
                pixel_number = len(tmp_arr[tmp_arr>r1_cut])

                if pixel_number > min_pixel:
                    event_container[keks+i] = event

In [ ]:
# histogram for number of pixels > r1 cut for 

event_container = {}
waveform_min_amplitude = 2
min_pixel = 200
pixel_number_arr = np.array([])
pixel_number_dict = {}
t1 = time.perf_counter(), time.process_time()



for k in range(1,41):
    t3 = time.perf_counter(), time.process_time()
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=1000)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        tmp_max_pixel_arr = np.array([])
        if len(event.trigger.tels_with_trigger) > 1:
            for tel_id in event.trigger.tels_with_trigger:
                tmp_arr = np.array([])
                for pix_id in event.r1.tel[tel_id].waveform:
                    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                #pixel_number = len(tmp_arr[tmp_arr>r1_cut])
                tmp_max_pixel_arr = np.append(tmp_max_pixel_arr,len(tmp_arr[tmp_arr>r1_cut])) 
            pixel_number_arr = np.append(pixel_number_arr, max(tmp_max_pixel_arr))
            if max(tmp_max_pixel_arr) > min_pixel:
                    event_container[keks+i] = event
    t2 = time.perf_counter(), time.process_time() 
    print(f" Real time of iteration: {t2[0] - t3[0]:.2f} seconds")
    print(f" CPU time of iteration: {t2[1] - t3[1]:.2f} seconds")

print(f" Final Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" Final CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
#print("len = ", len(sorted(pixel_number_arr[pixel_number_arr>200])))
#print(sorted(pixel_number_arr[pixel_number_arr>200]))
print(len(pixel_number_arr))

In [ ]:
fig = plt.figure(figsize = (10,9))
#ax = fig.add_subplot(1,1,1)


x,y,_ = plt.hist(pixel_number_arr, bins = 100, range = (0,1855), density = True, histtype = 'step', alpha = 1, label = 'Number of pixels with r1 amplitude > 2')

plt.xlim(0,1856)
plt.ylim(0,0.001)

#ax.set_xlim(0, 5)
#ax.set_ylim(0, 2)
#major_ticks_x = np.arange(0, 1855, 1)
#minor_ticks_x = np.arange(0, 1855, 1)
#major_ticks_y = np.arange(0, 5, 0.5)
#minor_ticks_y = np.arange(0, 5, 0.5)
#ax.set_xticks(major_ticks_x)
#ax.set_xticks(minor_ticks_x, minor=True)
#ax.set_yticks(major_ticks_y)
#ax.set_yticks(minor_ticks_y, minor=True)
# And a corresponding grid
#ax.grid(which='both')
# Or if you want different settings for the grids:
#ax.grid(which='minor', alpha=0.2)
#ax.grid(which='major', alpha=0.5)
print("max number of pixels per event = ", int(max(pixel_number_arr)))
print("number of events with number of pixels > 200 = ", len(sorted(pixel_number_arr[pixel_number_arr>200])))
plt.legend()
plt.xlabel("Number of pixels")
plt.ylabel("Probability density")
plt.grid()

In [ ]:
event_container_doubled = {}
waveform_min_amplitude = 2
min_pixel = 400
pixel_number_arr_doubled = np.array([])
pixel_number_dict = {}
t1 = time.perf_counter(), time.process_time()



for k in range(1,41):
    t3 = time.perf_counter(), time.process_time()
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename, max_events=1000)
    event_iterator = iter(source)
    keks = k*1000
    for i,event in enumerate(event_iterator):
        pixel_number = 0
        tel_counter = 0
        if len(event.trigger.tels_with_trigger) > 1:
            for tel_id in event.trigger.tels_with_trigger:
                tmp_arr = np.array([])
                for pix_id in event.r1.tel[tel_id].waveform:
                    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
                pixel_number += len(tmp_arr[tmp_arr>r1_cut])
                tel_counter += 1
                if tel_counter == 2:
                    pixel_number_arr_doubled = np.append(pixel_number_arr_doubled, pixel_number)
                    if pixel_number > min_pixel:
                        event_container_doubled[keks+i] = event
                    break
    t2 = time.perf_counter(), time.process_time() 
    print(f" Real time of iteration: {t2[0] - t3[0]:.2f} seconds")
    print(f" CPU time of iteration: {t2[1] - t3[1]:.2f} seconds")

print(f" Final Real time: {t2[0] - t1[0]:.2f} seconds")
print(f" Final CPU time: {t2[1] - t1[1]:.2f} seconds")

In [ ]:
max(pixel_number_arr_doubled)

In [ ]:
fig = plt.figure(figsize = (10,9))
#ax = fig.add_subplot(1,1,1)


x,y,_ = plt.hist(pixel_number_arr_doubled, bins = 100, range = (0,1855), density = True, histtype = 'step', alpha = 1, label = 'Number of pixels with r1 amplitude > 2')

plt.xlim(0,3711)
#plt.ylim(0,0.01)

#ax.set_xlim(0, 5)
#ax.set_ylim(0, 2)
#major_ticks_x = np.arange(0, 1855, 1)
#minor_ticks_x = np.arange(0, 1855, 1)
#major_ticks_y = np.arange(0, 5, 0.5)
#minor_ticks_y = np.arange(0, 5, 0.5)
#ax.set_xticks(major_ticks_x)
#ax.set_xticks(minor_ticks_x, minor=True)
#ax.set_yticks(major_ticks_y)
#ax.set_yticks(minor_ticks_y, minor=True)
# And a corresponding grid
#ax.grid(which='both')
# Or if you want different settings for the grids:
#ax.grid(which='minor', alpha=0.2)
#ax.grid(which='major', alpha=0.5)
print("max number of pixels per event = ", int(max(pixel_number_arr_doubled)))
#print("number of events with number of pixels > 400 = ", len(sorted(pixel_number_arr_doubled[pixel_number_arr>400])))
plt.legend()
plt.xlabel("Number of pixels")
plt.ylabel("Probability density")
plt.grid()

In [ ]:
len(event_container)

In [ ]:
pixel_number_array = []
true_image_sum = []
for i in event_container:
    for tel in event_container[i].trigger.tels_with_trigger:
        tmp_arr = np.array([])
        #print("Number of active pixels in the event = " np.count_nonzero(event_container[i].simulation.true_image))
        n_sim_pix = np.count_nonzero(event_container[i].simulation.tel[tel].true_image)
        for pix_id in event_container[i].r1.tel[tel].waveform:
            tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
        pixel_number = len(tmp_arr[tmp_arr>r1_cut])
        plt.figure(figsize = (9,6))
        camgeom = source.subarray.tel[tel].camera.geometry
        #title=f"Event_{key}_optEff{round(muon_events[key].muon.tel[tel].efficiency.optical_efficiency,3)}_width{round(muon_events[key].muon.tel[tel].efficiency.width.to_value(),3)}_telescope{tel}"
        title=f"Event{i} - number of simulated pixels = {n_sim_pix} and Pixels with r1 waveform > 2 = {pixel_number}"
        disp = CameraDisplay(camgeom,title=title)
        disp.image = event_container[i].simulation.tel[tel].true_image
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        pixel_number_array.append(pixel_number)
        true_image_sum.append(n_sim_pix)
        plt.show()

In [ ]:
energies = []
for i in event_container:
    energies.append(event_container[i].simulation.shower.energy.to_value())


In [ ]:
plt.plot(energies)
#energies
plt.ylabel("TeV")

In [ ]:
event_container[1294].simulation.tel[2].true_image

In [ ]:
event_container[1294].trigger.tels_with_trigger

In [ ]:
np.count_nonzero(event_container[1294].simulation.tel[2].true_image)

In [ ]:
r1_cut = 2
tmp_max_pixel_arr = np.array([])
tmp_arr = np.array([])
for pix_id in event_container[1294].r1.tel[2].waveform:
    tmp_arr = np.append(tmp_arr, np.max(pix_id)) # знаходить максимальне значення в кожному пікселі
pixel_number = len(tmp_arr[tmp_arr>r1_cut])
print(pixel_number)

In [ ]:
number_of_true_pixels = []
energies = []
for i in event_container:
    tmp = []
    print('new event')
    energies.append(event_container[i].simulation.shower.energy.to_value())
    for tel in event_container[i].trigger.tels_with_trigger:
        tmp.append(np.count_nonzero(event_container[i].simulation.tel[tel].true_image))
    number_of_true_pixels.append(max(tmp))

In [ ]:
x,y,z = plt.hist(number_of_true_pixels, alpha = 0.5, bins = 6, label = 'number of simulated pixels')
plt.ylabel("Count")
plt.xlabel("Number of pixels")
plt.grid()
plt.legend()

In [ ]:
x,y,z = plt.hist(energies, alpha = 0.5, bins = 10, label = 'Energies of event')
plt.ylabel("Count")
plt.xlabel("Energies [TeV]")
plt.grid()
plt.legend()

In [ ]:
max(number_of_true_pixels)

In [ ]:
len(energies)

In [ ]:
1+2

In [ ]:
return_pixels = np.array([])
island_id = np.array([1,2,2,2,3,4,4,5,5,6,1,1,1,1])
t = 1

test = []
return_pixels = np.append(return_pixels, np.where(island_id ==t)[0])

test.append(np.where(island_id ==t)[0])

In [ ]:
test

In [ ]:
np.where(island_id==2)[0]

In [ ]:
return_pixels

In [ ]:
test_arr = [1,2,3]

In [ ]:
test_list = [1,2,3,4,5]

result = ','.join(map(str, test_list))
with open("/Users/vdk/testtest.txt", 'a') as fout:
    fout.write(result)
    fout.write(',')

In [ ]:
result

In [ ]:
2000/60

In [ ]:
for i in range(0,10):
    print(f"srun 4LST_cleaning_cluster.py {i * 20} {i*20 + 20} \n")

In [ ]:
import joblib

In [ ]:
joblib.cpu_count()

In [ ]:
68/1356

In [ ]:
73/2873